In [2]:
# from google.colab import auth
# auth.authenticate_user()


In [3]:
!pip install pandas-gbq --quiet

import pandas as pd
from pandas_gbq import read_gbq

project_id = "pivotal-canto-466205-p6"
query = """
SELECT order_id,products
FROM `pivotal-canto-466205-p6.intent_inference.Orders`
WHERE dataset_id=1
"""

df = read_gbq(query, project_id=project_id)
df.head()


Downloading: 100%|██████████|


,order_id,products
0,1686389,"[Craft Ales, Diet Sodas, Soymilk, Oatmeal Mixe..."
1,1723267,"[Parmesan Cheese, Cleaning Solutions, Dairy Al..."
2,1730526,"[Small Hass Avocado, Sparkling Water Grapefrui..."
3,1772935,"[Small Hass Avocado, Vegan Honey Mustard Dress..."
4,1794821,"[Small Hass Avocado, Apples, Organic Strawberr..."


In [4]:
import ast

df['products'] = df['products'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [5]:
# Expand each product into separate rows
basket = df.explode('products')

# Create one-hot encoded matrix: rows = orders, columns = products
basket = basket.groupby(['order_id', 'products']).size().unstack(fill_value=0)

# Convert counts to 1/0
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

basket.head()


/tmp/ipython-input-521103163.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


products,0 Calorie Fuji Apple Pear Water Beverage,0% Fat Free Organic Milk,1 % Lowfat Milk,1 Liter,1% Chocolate Milk,1% Low Fat Cottage Cheese,1% Low Fat Milk,1% Lowfat Milk,1% Milk,1/3 Less Fat Cream Cheese,...,Zero Cake Batter,Zero Calorie Creamy Root Beer Stevia Soda,Zero Calorie Lemon Lime Soda,Zero Calorie Lemon Lime Sports Drink,Zero Calorie Lemon Lime Twist Soda,Zero Rise Orange,Zero Strawberry Lemon,Zinfandel,Zucchini Noodles,Zucchini Squash
order_id,,,,,,,,,,,,,,,,,,,,,
1684409,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1684410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1684422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1684433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1684456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Mine frequent itemsets
frequent_itemsets = fpgrowth(basket, min_support=0.01, use_colnames=True)

# Generate rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

print("Frequent Itemsets:\n", frequent_itemsets.head())
print("\nRules:\n", rules.head())


/usr/local/lib/python3.12/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: Dep

Frequent Itemsets:
    support                       itemsets
0   0.2466           (Dairy Alternatives)
1   0.0362              (Bathroom Tissue)
2   0.0198               (Organic Banana)
3   0.0106  (Organic Reduced Fat 2% Milk)
4   0.1274                         (Eggs)

Rules:
                   antecedents           consequents  antecedent support  \
0           (Bathroom Tissue)  (Dairy Alternatives)              0.0362   
1                      (Eggs)  (Dairy Alternatives)              0.1274   
2  (Dairy Alternatives, Eggs)       (Citrus Fruits)              0.0412   
3       (Citrus Fruits, Eggs)  (Dairy Alternatives)              0.0346   
4      (Frozen Berries, Eggs)       (Citrus Fruits)              0.0234   

   consequent support  support  confidence      lift  representativity  \
0              0.2466   0.0110    0.303867  1.232228               1.0   
1              0.2466   0.0412    0.323391  1.311399               1.0   
2              0.2156   0.0146    0.354369  1.

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [7]:
print("\nRules:\n", rules)



Rules:
                     antecedents           consequents  antecedent support  \
0             (Bathroom Tissue)  (Dairy Alternatives)              0.0362   
1                        (Eggs)  (Dairy Alternatives)              0.1274   
2    (Dairy Alternatives, Eggs)       (Citrus Fruits)              0.0412   
3         (Citrus Fruits, Eggs)  (Dairy Alternatives)              0.0346   
4        (Frozen Berries, Eggs)       (Citrus Fruits)              0.0234   
..                          ...                   ...                 ...   
171              (Fruit Snacks)       (Citrus Fruits)              0.0314   
172              (Fruit Snacks)  (Dairy Alternatives)              0.0314   
173          (Organic Cucumber)              (Apples)              0.0370   
174          (Organic Cucumber)  (Dairy Alternatives)              0.0370   
175          (Organic Cucumber)       (Citrus Fruits)              0.0370   

     consequent support  support  confidence      lift  representa

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [8]:
rules.to_csv("rules for insta_cart.csv", index=False)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [9]:
# Sort by lift in descending order
rules_sorted = rules.sort_values(by="lift", ascending=False)

# Show top 10 rules with highest lift
print(rules_sorted.head(10))

                                    antecedents       consequents  \
151                      (Organic Yellow Onion)   (Spicy Peppers)   
89                                      (Herbs)   (Spicy Peppers)   
40      (Cruciferous Vegetables, Citrus Fruits)          (Apples)   
16               (Organic Hass Avocado, Apples)   (Citrus Fruits)   
15        (Organic Hass Avocado, Citrus Fruits)          (Apples)   
109                  (Apples, Citrus Beverages)   (Citrus Fruits)   
112                       (Organic Blueberries)  (Frozen Berries)   
102       (Citrus Fruits, Organic Strawberries)          (Apples)   
160                            (Pretzel Snacks)     (Snack Chips)   
104  (Dairy Alternatives, Organic Strawberries)          (Apples)   

     antecedent support  consequent support  support  confidence      lift  \
151              0.0326              0.0994   0.0110    0.337423  3.394601   
89               0.0664              0.0994   0.0210    0.316265  3.181741   
40    

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag